# Task 4.1 Demonstrate execution Mode

## Objective 1 Execute on hardware

run your circuits on hardware and produce the outputs of the quantum computation

Guide:
* Run with Primitives
* Configure runtime options
* execution modes
* manage jobs
* qpu and platform information

## Objective 2 execution modes

* Job mode : single request of sampler or estimator, specify mode=backend when you initialize the primitive
* Batch mode: multi-job workload made up of independently executable jobs, specify mode=batch
* Session mode: dedicated window to run a multi-job workload , used for variational algorithms, specify mode=session